In [4]:
import pandas as pd
import os
import glob

# Define path to data_header relative to this notebook
# Notebook is in: .../Test_code/OpenMarket/251209_NonVip/
# data_header is in: .../Test_code/data_header/
data_header_path = "../../data_header"

# Output directory is the current directory
output_dir = "."

# Check if data_header exists
if os.path.exists(data_header_path):
    # Get all subdirectories in data_header
    subdirs = [d for d in os.listdir(data_header_path) if os.path.isdir(os.path.join(data_header_path, d))]
    subdirs.sort()
    
    print(f"Found {len(subdirs)} folders to process.")

    for folder in subdirs:
        folder_path = os.path.join(data_header_path, folder)
        
        # Get all csv files in the folder
        csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
        
        if not csv_files:
            print(f"Skipping {folder}: No CSV files found.")
            continue
            
        print(f"\nProcessing {folder}...")
        
        dfs = []
        for file in csv_files:
            try:
                # Read CSV
                df = pd.read_csv(file)
                dfs.append(df)
            except Exception as e:
                print(f"  Error reading {os.path.basename(file)}: {e}")
        
        if dfs:
            # Concatenate all dataframes vertically
            merged_df = pd.concat(dfs, ignore_index=True)
            
            # Determine output filename
            # Logic: '01.회원정보_헤드' -> 'Merged_회원정보.csv'
            
            # Remove the number prefix (e.g. "01.") if present
            name_part = folder
            if '.' in name_part:
                parts = name_part.split('.', 1)
                if parts[0].isdigit():
                    name_part = parts[1]
            
            # Remove "_헤드" suffix if present
            name_part = name_part.replace("_헤드", "")
            
            output_filename = f"Merged_{name_part}.csv"
            output_path = os.path.join(output_dir, output_filename)
            
            # Save to CSV
            merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
            
            print(f"  Merged {len(dfs)} files into '{output_filename}' ({len(merged_df)} rows)")
else:
    print(f"Directory not found: {data_header_path}")

Found 8 folders to process.

Processing 01.회원정보_헤드...
  Merged 6 files into 'Merged_회원정보.csv' (60000 rows)

Processing 02.신용정보_헤드...
  Merged 6 files into 'Merged_신용정보.csv' (60000 rows)

Processing 03.승인매출정보_헤드...
  Merged 6 files into 'Merged_승인매출정보.csv' (60000 rows)

Processing 04.청구정보_헤드...
  Merged 6 files into 'Merged_청구정보.csv' (60000 rows)

Processing 05.잔액정보_헤드...
  Merged 6 files into 'Merged_잔액정보.csv' (60000 rows)

Processing 06.채널정보_헤드...
  Merged 6 files into 'Merged_채널정보.csv' (60000 rows)

Processing 07.마케팅정보_헤드...
  Merged 6 files into 'Merged_마케팅정보.csv' (60000 rows)

Processing 08.성과정보_헤드...
  Merged 6 files into 'Merged_성과정보.csv' (60000 rows)


In [6]:
df = pd.read_csv("./Final_merged_all_data.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 898 entries, 기준년월 to 최종카드발급경과월
dtypes: float64(62), int64(797), object(39)
memory usage: 411.1+ MB


In [8]:
columns_drop = [
    # ---------------------------------------------------------
    # 01. 회원정보 제거 대상
    # ---------------------------------------------------------
    "회원여부_이용가능_CA", "회원여부_이용가능_카드론", "소지카드수_유효_신용", "소지카드수_이용가능_신용", 
    "입회일자_신용", "동의여부_한도증액안내", "수신거부여부_TM", 
    "수신거부여부_DM", "수신거부여부_메일", "수신거부여부_SMS", "가입통신회사코드", "최종탈회후경과월", 
    "탈회횟수_발급6개월이내", "탈회횟수_발급1년이내", "직장시도명", "마케팅동의여부", 
    "유효카드수_신용_가족", "유효카드수_체크_가족", "이용가능카드수_신용_가족", "이용가능카드수_체크_가족", 
    "이용카드수_신용_가족", "이용카드수_체크_가족", "이용금액_R3M_신용", "이용금액_R3M_신용_가족", 
    "이용금액_R3M_체크", "이용금액_R3M_체크_가족", "_1순위카드이용금액", "_1순위카드이용건수", 
    "_1순위신용체크구분", "_2순위카드이용금액", "_2순위카드이용건수", "_2순위신용체크구분", 
    "최종유효년월_신용_이용가능", "최종유효년월_신용_이용", "최종카드발급일자", "보유여부_해외겸용_본인", 
    "이용가능여부_해외겸용_본인", "이용여부_3M_해외겸용_본인", "보유여부_해외겸용_신용_본인", 
    "이용가능여부_해외겸용_신용_본인", "이용여부_3M_해외겸용_신용_본인", "연회비발생카드수_B0M", 
    "연회비할인카드수_B0M", "기본연회비_B0M", "제휴연회비_B0M", "할인금액_기본연회비_B0M", 
    "할인금액_제휴연회비_B0M", "청구금액_기본연회비_B0M", "청구금액_제휴연회비_B0M", 
    "상품관련면제카드수_B0M", "임직원면제카드수_B0M", "우수회원면제카드수_B0M", "기타면제카드수_B0M", 
    "카드신청건수", "최종카드발급경과월",

    # ---------------------------------------------------------
    # 02. 신용 정보 제거 대상
    # ---------------------------------------------------------
    '최초한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 
    'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV약정청구율', 
    'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', 
    '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', 
    '한도증액후경과월', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 
    '일시불ONLY전환가능여부', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', 
    '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', 
    '시장단기연체여부_R3M', '시장연체상환여부_R6M', '시장연체상환여부_R3M',

    # ---------------------------------------------------------
    # 03. 승인매출 정보 (기본 집계)
    # ---------------------------------------------------------
    '최종이용일자_신판', '최종이용일자_CA', '최종이용일자_일시불', '최종이용일자_할부',
    '이용건수_신판_B0M', '이용건수_일시불_B0M', '이용건수_할부_B0M', '이용건수_할부_유이자_B0M',
    '이용건수_할부_무이자_B0M', '이용건수_부분무이자_B0M', '이용건수_CA_B0M', '이용건수_체크_B0M',
    '이용건수_카드론_B0M', '이용금액_신판_B0M', '이용금액_일시불_B0M', '이용금액_할부_B0M',
    '이용금액_할부_유이자_B0M', '이용금액_할부_무이자_B0M', '이용금액_부분무이자_B0M', '이용금액_CA_B0M',
    '이용금액_체크_B0M', '이용금액_카드론_B0M', '이용후경과월_신판', '이용후경과월_일시불',
    '이용후경과월_할부', '이용후경과월_할부_유이자', '이용후경과월_할부_무이자', '이용후경과월_부분무이자',
    '이용후경과월_CA', '이용후경과월_체크', '이용후경과월_카드론', '이용건수_신판_R12M',
    '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_할부_유이자_R12M', '이용건수_할부_무이자_R12M',
    '이용건수_부분무이자_R12M', '이용건수_CA_R12M', '이용건수_체크_R12M', '이용건수_카드론_R12M',
    '이용금액_신판_R12M', '이용금액_일시불_R12M', '이용금액_할부_R12M', '이용금액_할부_유이자_R12M',
    '이용금액_할부_무이자_R12M', '이용금액_부분무이자_R12M', '이용금액_CA_R12M', '이용금액_체크_R12M',
    '이용금액_카드론_R12M', '이용개월수_신판_R12M', '이용개월수_일시불_R12M', '이용개월수_할부_R12M',
    '이용개월수_할부_유이자_R12M', '이용개월수_할부_무이자_R12M', '이용개월수_부분무이자_R12M', '이용개월수_CA_R12M',
    '이용개월수_체크_R12M', '이용개월수_카드론_R12M', '이용건수_신판_R6M', '이용건수_일시불_R6M',
    '이용건수_할부_R6M', '이용건수_할부_유이자_R6M', '이용건수_할부_무이자_R6M', '이용건수_부분무이자_R6M',
    '이용건수_CA_R6M', '이용건수_체크_R6M', '이용건수_카드론_R6M', '이용금액_신판_R6M',
    '이용금액_일시불_R6M', '이용금액_할부_R6M', '이용금액_할부_유이자_R6M', '이용금액_할부_무이자_R6M',
    '이용금액_부분무이자_R6M', '이용금액_CA_R6M', '이용금액_체크_R6M', '이용금액_카드론_R6M',
    '이용개월수_신판_R6M', '이용개월수_일시불_R6M', '이용개월수_할부_R6M', '이용개월수_할부_유이자_R6M',
    '이용개월수_할부_무이자_R6M', '이용개월수_부분무이자_R6M', '이용개월수_CA_R6M', '이용개월수_체크_R6M',
    '이용개월수_카드론_R6M', '이용건수_신판_R3M', '이용건수_일시불_R3M', '이용건수_할부_R3M',
    '이용건수_할부_유이자_R3M', '이용건수_할부_무이자_R3M', '이용건수_부분무이자_R3M', '이용건수_CA_R3M',
    '이용건수_체크_R3M', '이용건수_카드론_R3M', '이용금액_신판_R3M', '이용금액_일시불_R3M',
    '이용금액_할부_R3M', '이용금액_할부_유이자_R3M', '이용금액_할부_무이자_R3M', '이용금액_부분무이자_R3M',
    '이용금액_CA_R3M', '이용금액_체크_R3M', '이용금액_카드론_R3M', '이용개월수_신판_R3M',
    '이용개월수_일시불_R3M', '이용개월수_할부_R3M', '이용개월수_할부_유이자_R3M', '이용개월수_할부_무이자_R3M',
    '이용개월수_부분무이자_R3M', '이용개월수_CA_R3M', '이용개월수_체크_R3M', '이용개월수_카드론_R3M',
    '이용가맹점수', '납부_통신비이용금액', '납부_관리비이용금액', '납부_렌탈료이용금액', '납부_가스전기료이용금액',
    '납부_보험료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '납부_기타이용금액',
    '_1순위납부업종', '_1순위납부업종_이용금액', '_2순위납부업종', '_2순위납부업종_이용금액',
    '_3순위납부업종', '_3순위납부업종_이용금액',

    # ---------------------------------------------------------
    # 03. 승인매출 정보 (세부 내역: 217~430 구간)
    # ---------------------------------------------------------
    # [217~247] 할부 세부 데이터 (3개월 이하/유이자/무이자/부분무이자 등 세분화된 항목 제외)
    '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M',
    '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M',
    '할부건수_유이자_6M_R12M', '할부건수_유이자_12M_R12M', '할부건수_유이자_14M_R12M',
    '할부금액_유이자_6M_R12M', '할부금액_유이자_12M_R12M', '할부금액_유이자_14M_R12M',
    '할부건수_무이자_6M_R12M', '할부건수_무이자_12M_R12M', '할부건수_무이자_14M_R12M',
    '할부금액_무이자_6M_R12M', '할부금액_무이자_12M_R12M', '할부금액_무이자_14M_R12M',
    '할부건수_부분_6M_R12M', '할부건수_부분_12M_R12M', '할부건수_부분_14M_R12M',
    '할부금액_부분_6M_R12M', '할부금액_부분_12M_R12M', '할부금액_부분_14M_R12M',

    # [248~286] RP(정기결제) 비주력/세부 변동 지표 (학습비, 유선방송 등 상대적 중요도 낮은 항목)
    'RP건수_제휴사서비스직접판매_B0M', 'RP건수_학습비_B0M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP건수_교통_B0M',
    'RP후경과월_제휴사서비스직접판매', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험',
    'RP후경과월_학습비', 'RP후경과월_유선방송', 'RP후경과월_건강', 'RP후경과월_교통',
    '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월',
    '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월',
    '증감_RP건수_학습비_전월', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '증감_RP건수_교통_전월',

    # [287~298] 장기카드대출 일부 (회의록 기준)
    '최종카드론이용경과월', '카드론이용월수_누적',

    # [299~395] 타사 페이(Pay) 상세 내역 (A~D 페이 등 개별 페이 내역 제외)
    '이용개월수_A페이_R6M', '이용개월수_B페이_R6M', '이용개월수_C페이_R6M', '이용개월수_D페이_R6M',
    '이용금액_A페이_R6M', '이용금액_B페이_R6M', '이용금액_C페이_R6M', '이용금액_D페이_R6M',
    '이용건수_A페이_R6M', '이용건수_B페이_R6M', '이용건수_C페이_R6M', '이용건수_D페이_R6M',
    '이용금액_A페이_R3M', '이용금액_B페이_R3M', '이용금액_C페이_R3M', '이용금액_D페이_R3M',
    '이용건수_A페이_R3M', '이용건수_B페이_R3M', '이용건수_C페이_R3M', '이용건수_D페이_R3M',
    '이용금액_A페이_B0M', '이용금액_B페이_B0M', '이용금액_C페이_B0M', '이용금액_D페이_B0M',
    '이용건수_A페이_B0M', '이용건수_B페이_B0M', '이용건수_C페이_B0M', '이용건수_D페이_B0M',

    # [396~397] 가맹점 매출 과거 데이터
    '가맹점매출금액_B1M', '가맹점매출금액_B2M',

    # [410~413] 할부 전환 이력
    '건수_할부전환_R6M', '건수_할부전환_R3M', '금액_할부전환_R6M', '금액_할부전환_R3M',

    # [414~415] 무실적/유실적 기간 (파생변수로 대체 가능하거나 중요도 낮음)
    '연속무실적개월수_기본_24M_카드', '연속유실적개월수_기본_24M_카드',

    # [416~419] ATM 이용 (단순 현금 인출/대출은 제외)
    '신청건수_ATM_CA_B0', '신청건수_ATM_CL_B0', '신청건수_ATM_CA_R6M', '신청건수_ATM_CL_R6M',

    # [420~429] 승인 거절 (입력 오류 등 단순 실수 제외)
    '승인거절건수_입력오류_B0M', '승인거절건수_기타_B0M', '승인거절건수_입력오류_R3M', '승인거절건수_기타_R3M',

    # ---------------------------------------------------------
    # 04. 청구입금 정보 제거 대상
    # ---------------------------------------------------------
    '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', 
    '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', 
    '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', 
    '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', 
    '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', 
    '포인트_포인트_월적립_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', 
    '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', 
    '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', 
    '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', 
    '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', 
    '연체건수_R3M',

    # ---------------------------------------------------------
    # 06. 채널 정보 제거 대상
    # ---------------------------------------------------------
    '이용메뉴건수_ARS_R6M', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS',
    '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문횟수_PC_R6M', '방문일수_PC_R6M',
    '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문월수_앱_R6M', '방문횟수_모바일웹_R6M',
    '방문일수_모바일웹_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M',
    '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '방문횟수_모바일웹_B0M',
    '방문일수_모바일웹_B0M', '인입횟수_금융_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M',
    '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M',
    '인입불만일수_IB_R6M', '인입불만월수_IB_R6M', '인입불만후경과월_IB_R6M', '인입불만일수_IB_B0M',
    '상담건수_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_선결제_B0M', 'IB문의건수_결제_B0M',
    'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M',
    'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M',
    'IB문의건수_포인트_B0M', 'IB문의건수_카드발급_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M',
    'IB문의건수_CA_B0M', 'IB문의건수_CL_RV_B0M', 'IB문의건수_CS_B0M', 'IB상담건수_VOC_B0M',
    'IB상담건수_VOC민원_B0M', 'IB상담건수_VOC불만_B0M', 'IB상담건수_금감원_B0M', '상담건수_R6M',
    'IB문의건수_사용승인내역_R6M', 'IB문의건수_한도_R6M', 'IB문의건수_선결제_R6M', 'IB문의건수_결제_R6M',
    'IB문의건수_할부_R6M', 'IB문의건수_정보변경_R6M', 'IB문의건수_결제일변경_R6M', 'IB문의건수_명세서_R6M',
    'IB문의건수_비밀번호_R6M', 'IB문의건수_SMS_R6M', 'IB문의건수_APP_R6M', 'IB문의건수_부대서비스_R6M',
    'IB문의건수_포인트_R6M', 'IB문의건수_카드발급_R6M', 'IB문의건수_BL_R6M', 'IB문의건수_분실도난_R6M',
    'IB문의건수_CA_R6M', 'IB문의건수_CL_RV_R6M', 'IB문의건수_CS_R6M', 'IB상담건수_VOC_R6M',
    'IB상담건수_VOC민원_R6M', 'IB상담건수_VOC불만_R6M', 'IB상담건수_금감원_R6M', '불만제기건수_B0M',
    '불만제기건수_R12M', '불만제기후경과월_R12M', '당사PAY_방문횟수_B0M', '당사PAY_방문횟수_R6M',
    '당사PAY_방문월수_R6M', '당사멤버쉽_방문횟수_B0M', '당사멤버쉽_방문횟수_R6M', '당사멤버쉽_방문월수_R6M',
    'OS구분코드', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R6M', '홈페이지_금융건수_R3M',
    '홈페이지_선결제건수_R3M'
]


In [9]:
len(columns_drop)

438

In [11]:
column_list = df.columns

In [14]:
columns_filtered = column_list.drop(columns_drop)

In [17]:
filtered_df = pd.DataFrame(df[columns_filtered])

In [18]:
filtered_df.to_csv('필터된 데이터.csv', index=False)

In [20]:
dropped_df = pd.DataFrame(df[columns_drop])

In [21]:
dropped_df.to_csv("드랍된 데이터.csv", index=False)